<a href="https://colab.research.google.com/github/Heity94/01_DataScience_2021/blob/main/TWSM/Class_notes_PH/class6_PH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 17.5 MB/s 
     |████████████████████████████████| 596 kB 47.0 MB/s 
     |████████████████████████████████| 84 kB 2.0 MB/s 
     |████████████████████████████████| 6.6 MB 19.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


https://huggingface.co/course/chapter1/3?fw=pt

In [2]:
import numpy as np
#possibly replace with e.g. util.pytorch_cos_sim from sentence_transformers
from scipy.spatial.distance import cosine

from transformers import pipeline


In [3]:
#this will download the default gpt2 model, around 1.2GB (or just 530MB??)
generator = pipeline("text-generation")
generator("In this course, we will teach you how to")

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "In this course, we will teach you how to create the next amazing VR headset from scratch in VR applications like Vive or Oculus Rift, including an early stage VR demo, and get you looking forward to the future of gaming. You'll learn everything you"}]

Mask filling

In [4]:
unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

[{'score': 0.196198508143425,
  'sequence': 'This course will teach you all about mathematical models.',
  'token': 30412,
  'token_str': ' mathematical'},
 {'score': 0.040527332574129105,
  'sequence': 'This course will teach you all about computational models.',
  'token': 38163,
  'token_str': ' computational'}]

### Tokenizer and models

DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. It has 40% less parameters than bert-base-uncased, runs 60% faster while preserving over 95% of BERT’s performances as measured on the GLUE language understanding benchmark.

https://huggingface.co/docs/transformers/model_doc/distilbert

The abstract from the paper is the following:

> As Transfer Learning from large-scale pre-trained models becomes more prevalent in Natural Language Processing (NLP), operating these large models in on-the-edge and/or under constrained computational training or inference budgets remains challenging. In this work, we propose a method to pre-train a smaller general-purpose language representation model, called DistilBERT, which can then be fine-tuned with good performances on a wide range of tasks like its larger counterparts. While most prior work investigated the use of distillation for building task-specific models, we leverage knowledge distillation during the pretraining phase and show that it is possible to reduce the size of a BERT model by 40%, while retaining 97% of its language understanding capabilities and being 60% faster. To leverage the inductive biases learned by larger models during pretraining, we introduce a triple loss combining language modeling, distillation and cosine-distance losses. Our smaller, faster and lighter model is cheaper to pre-train and we demonstrate its capabilities for on-device computations in a proof-of-concept experiment and a comparative on-device study.

In [5]:
from transformers import AutoTokenizer

#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}


In [6]:
#What about slang akronyms ?
IDs = tokenizer(["OMG, this movie", "WTF is going"], padding=True, truncation=True, return_tensors="tf")
print(IDs)
print(tokenizer.tokenize("OMG"))
print(tokenizer.tokenize("LOL"))
print(tokenizer.tokenize("BERT"))
print(tokenizer.tokenize("Interdisciplinary"))
print(tokenizer.tokenize("comprehensibility"))

#notice that the full BERT tokenizer would split the following into sub words:
print(tokenizer.tokenize("transformers"))

{'input_ids': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[  101, 18168,  2290,  1010,  2023,  3185,   102],
       [  101,  1059, 24475,  2003,  2183,   102,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0]], dtype=int32)>}
['om', '##g']
['lo', '##l']
['bert']
['interdisciplinary']
['com', '##pre', '##hen', '##sibility']
['transformers']


Notice the identical tokens for identical words

In [7]:
raw_inputs = [
    "How does the dimension of the embedding come about",
    "How do you come around?",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[ 101, 2129, 2515, 1996, 9812, 1997, 1996, 7861, 8270, 4667, 2272,
        2055,  102],
       [ 101, 2129, 2079, 2017, 2272, 2105, 1029,  102,    0,    0,    0,
           0,    0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]], dtype=int32)>}


Notice the different ways of calling the tokenizer:

In [8]:
s = "BIPM alumni are in high demand!"
tokens = tokenizer.tokenize(s)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

ids2 = tokenizer(s, padding=True, truncation=True, return_tensors="tf")
print(ids2)

[12170, 9737, 9441, 2024, 1999, 2152, 5157, 999]
{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  101, 12170,  9737,  9441,  2024,  1999,  2152,  5157,   999,
          102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


#### Decoding tokens

Note that the decode method not only converts the indices back to tokens, but also groups together the tokens that were part of the same words to produce a readable sentence. 

In [9]:
decoded_string = tokenizer.decode(ids)
print(decoded_string)

bipm alumni are in high demand!


### Embedding

In [10]:
from transformers import TFAutoModel

#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
checkpoint = "bert-base-uncased"
model = TFAutoModel.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [11]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape)

(2, 13, 768)


So the output is high-dimensional (embedding of dimension 768 for each individual word!)

Let's find out if the embedding is really contextual, i.e. whether identical single words are different depending on the sentence:

In [12]:

How_embed_1 = outputs.last_hidden_state[0,0,:]
How_embed_2 = outputs.last_hidden_state[1,0,:]
print(np.sum(How_embed_1==How_embed_2))#0
#outputs.last_hidden_state[0:2,0,0:10]


print("similarity:")
print( 1 - cosine(How_embed_1, How_embed_2) ) 
#most_similar=data_

0
similarity:
0.9125811457633972


### Tasks:

1. Embedding Similarity 
  * Write a function that takes as input two sentences, finds identical words, computes their contextual embeddings and prints the cosine similarity.
  * Ask interesting questions (case, punctuation,...)
  * Use longer words and find similarity between sub words

2. Load the IMBD reviews and sample 500 positive and negative reviews. 
  * Extract the embedding vectors
  * Fit a Naive Bayes classifier

In [43]:
def similarity_common_words(sentence1, sentence2): 
  raw_inputs = [
    sentence1,
    sentence2,
]
  words_in_common = list(set(sentence1.lower().split()).intersection(sentence2.lower().split()))
  inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
  outputs = model(inputs)
  index_sentence1 = [sentence1.lower().split().index(i) for i in words_in_common]
  index_sentence2 = [sentence2.lower().split().index(i) for i in words_in_common]
  for i, word in enumerate(words_in_common):
    How_embed_1 = outputs.last_hidden_state[0,index_sentence1[i],:]
    How_embed_2 = outputs.last_hidden_state[1,index_sentence2[i],:]
    print("Similarity:", word)
    print(1 - cosine(How_embed_1, How_embed_2)) 

In [42]:
similarity_common_words("work is live", "live is work")

work
0.10128681361675262
live
0.22911491990089417
is
0.7664701342582703


In [46]:
similarity_common_words("I do not care", "I do care")

Similarity: i
0.9537034034729004
Similarity: do
0.855302631855011
Similarity: care
0.5524779558181763


In [28]:
cm_words = list(set([sentence.lower().split() for sentence in raw_inputs][0]).intersection([sentence.lower().split() for sentence in raw_inputs][1]))

In [30]:
sentences = [sentence.lower().split() for sentence in raw_inputs]

['come', 'how']

In [37]:
[sentences[0].index(i) for i in cm_words]

[7, 0]

In [38]:
cm_words

['come', 'how']

In [ ]:
sentence1, sentence2

sentence1.split().lower()

In [13]:
from google.colab import drive
drive.mount('/content/drive')

import pickle

# Import dataset
TWSM_path="/content/drive/MyDrive/teaching/TWSM/"
decoded_reviews = pickle.load(open(TWSM_path + "data/IMDB/decoded_reviews_train.pkl", "rb"))
train_labels = pickle.load(open(TWSM_path + "data/IMDB/train_labels.pkl", "rb"))


Mounted at /content/drive


FileNotFoundError: ignored